In [1]:
import webbrowser, os
import json
import boto3
import io
from io import BytesIO
import sys
from pprint import pprint
import pdf2image
import os

In [2]:
def extract_jpg(file_name):
    table_csv = get_table_csv_results(file_name)
    with open(output_file, "w") as fout:
        fout.write(table_csv)
    
    print('CSV OUTPUT FILE: ', output_file)
    
#extract_jpg('out.jpg')

In [14]:
def extract_pdf(file_name):    
    from pdf2image import convert_from_path
    pages = convert_from_path(file_name, 500)
    table_csv=""
    for page in pages:
        page.save('out.jpeg', 'JPEG')
        table_csv+= get_table_csv_results('out.png')
        os.remove("out.jpeg")
         
    with open(output_file, "w") as fout:
        fout.write(table_csv)
        
    print('CSV OUTPUT FILE: ', output_file)
    
        
#extract_pdf(file_name)    
    

In [4]:
def get_rows_columns_map(table_result, blocks_map):
    rows = {}
    for relationship in table_result['Relationships']:
        if relationship['Type'] == 'CHILD':
            for child_id in relationship['Ids']:
                cell = blocks_map[child_id]
                if cell['BlockType'] == 'CELL':
                    row_index = cell['RowIndex']
                    col_index = cell['ColumnIndex']
                    if row_index not in rows:
                        # create new row
                        rows[row_index] = {}
                        
                    # get the text value
                    rows[row_index][col_index] = get_text(cell, blocks_map)
    return rows


def get_text(result, blocks_map):
    text = ''
    if 'Relationships' in result:
        for relationship in result['Relationships']:
            if relationship['Type'] == 'CHILD':
                for child_id in relationship['Ids']:
                    word = blocks_map[child_id]
                    if word['BlockType'] == 'WORD':
                        text += word['Text'] + ' '
                    if word['BlockType'] == 'SELECTION_ELEMENT':
                        if word['SelectionStatus'] =='SELECTED':
                            text +=  'X '    
    return text


def get_table_csv_results(file_name):

    with open(file_name, 'rb') as file:
        img_test = file.read()
        bytes_test = bytearray(img_test)
        print('Image loaded', file_name)

    # process using image bytes
    # get the results
    client = boto3.client('textract')

    response = client.analyze_document(Document={'Bytes': bytes_test}, FeatureTypes=['TABLES'])

    # Get the text blocks
    blocks=response['Blocks']
    pprint(blocks)

    blocks_map = {}
    table_blocks = []
    for block in blocks:
        blocks_map[block['Id']] = block
        if block['BlockType'] == "TABLE":
            table_blocks.append(block)

    if len(table_blocks) <= 0:
        return "<b> NO Table FOUND </b>"

    csv = ''
    for index, table in enumerate(table_blocks):
        csv += generate_table_csv(table, blocks_map, index +1)
        csv += '\n\n'

    return csv

def generate_table_csv(table_result, blocks_map, table_index):
    rows = get_rows_columns_map(table_result, blocks_map)

    table_id = 'Table_' + str(table_index)
    
    # get cells.
    csv = 'Table: {0}\n\n'.format(table_id)

    for row_index, cols in rows.items():
        
        for col_index, text in cols.items():
            csv += '{}'.format(text) + ","
        csv += '\n'
        
    csv += '\n\n\n'
    return csv

In [18]:
file_name = 'demo.pdf'
output_file = file_name[:file_name.index('.')] + '.csv'

In [19]:
if(file_name[-3:]=="pdf"):
    extract_pdf(file_name)
else:
    extract_jpg(file_name)

Image loaded out.png
[{'BlockType': 'PAGE',
  'Geometry': {'BoundingBox': {'Height': 1.0,
                               'Left': 0.0,
                               'Top': 0.0,
                               'Width': 1.0},
               'Polygon': [{'X': 0.0, 'Y': 0.0},
                           {'X': 1.0, 'Y': 0.0},
                           {'X': 1.0, 'Y': 1.0},
                           {'X': 0.0, 'Y': 1.0}]},
  'Id': '8f57129b-ba39-4ee6-b852-6099542151a9',
  'Relationships': [{'Ids': ['79f66431-a956-4d0d-966a-d29c41385c53',
                             'ccabd443-5359-4bcb-a208-55123d1d8d88',
                             '6c1ce160-b11a-4ab6-bb90-7bed8ced715e',
                             'acc154f7-b9c3-43ec-a076-a55c58f2da38',
                             'd81118c9-a05c-47a9-b4f6-ce4369067d15',
                             'dda56003-52b6-4da8-b260-9e3348f01d30',
                             '0cd74156-3327-4df2-bbb4-dc9bd679cf43',
                             'de8e6cc1-453f-444a

               'Polygon': [{'X': 0.17740416526794434, 'Y': 0.7065368890762329},
                           {'X': 0.20160406827926636, 'Y': 0.7065368890762329},
                           {'X': 0.20160406827926636, 'Y': 0.7167229056358337},
                           {'X': 0.17740416526794434,
                            'Y': 0.7167229056358337}]},
  'Id': 'b817711e-96ec-4a54-b8f6-09fc544322ea',
  'Relationships': [{'Ids': ['9b235bb6-2e69-42c9-9af8-fc9912e8127d'],
                     'Type': 'CHILD'}],
  'Text': '7.5'},
 {'BlockType': 'LINE',
  'Confidence': 99.92434692382812,
  'Geometry': {'BoundingBox': {'Height': 0.01440634299069643,
                               'Left': 0.22018307447433472,
                               'Top': 0.7057599425315857,
                               'Width': 0.1416139453649521},
               'Polygon': [{'X': 0.22018307447433472, 'Y': 0.7057599425315857},
                           {'X': 0.3617970049381256, 'Y': 0.7057599425315857},
                

  'Id': '6df0d623-323d-41ea-a647-312e4348c778',
  'Text': 'land'},
 {'BlockType': 'WORD',
  'Confidence': 96.32415008544922,
  'Geometry': {'BoundingBox': {'Height': 0.012418420054018497,
                               'Left': 0.21860411763191223,
                               'Top': 0.2483140528202057,
                               'Width': 0.1210702583193779},
               'Polygon': [{'X': 0.21860411763191223, 'Y': 0.2483140528202057},
                           {'X': 0.3396743834018707, 'Y': 0.2483140528202057},
                           {'X': 0.3396743834018707, 'Y': 0.2607324719429016},
                           {'X': 0.21860411763191223,
                            'Y': 0.2607324719429016}]},
  'Id': 'e32694b8-6db4-44cc-8096-18b9d7c21f32',
  'Text': 'infrastructure,'},
 {'BlockType': 'WORD',
  'Confidence': 99.88798522949219,
  'Geometry': {'BoundingBox': {'Height': 0.014241100288927555,
                               'Left': 0.4065573513507843,
                           

                               'Left': 0.4714197516441345,
                               'Top': 0.6356343626976013,
                               'Width': 0.02435234934091568},
               'Polygon': [{'X': 0.4714197516441345, 'Y': 0.6356343626976013},
                           {'X': 0.4957720935344696, 'Y': 0.6356343626976013},
                           {'X': 0.4957720935344696, 'Y': 0.6462672352790833},
                           {'X': 0.4714197516441345, 'Y': 0.6462672352790833}]},
  'Id': '71e38d95-da56-4ef7-9a01-74dc1173c250',
  'Text': 'US'},
 {'BlockType': 'WORD',
  'Confidence': 99.52088165283203,
  'Geometry': {'BoundingBox': {'Height': 0.010921250097453594,
                               'Left': 0.49977293610572815,
                               'Top': 0.6353944540023804,
                               'Width': 0.05859532952308655},
               'Polygon': [{'X': 0.49977293610572815, 'Y': 0.6353944540023804},
                           {'X': 0.5583682656288147, 'Y':

                           {'X': 0.20923098921775818, 'Y': 0.22790053486824036},
                           {'X': 0.1726711243391037,
                            'Y': 0.22790053486824036}]},
  'Id': '83bcd7a0-081b-4ab0-90fc-8cd58f355f45',
  'Relationships': [{'Ids': ['e49d289c-ab10-4315-b416-89f881a83721'],
                     'Type': 'CHILD'}],
  'RowIndex': 1,
  'RowSpan': 1},
 {'BlockType': 'CELL',
  'ColumnIndex': 2,
  'ColumnSpan': 1,
  'Confidence': 99.99972534179688,
  'Geometry': {'BoundingBox': {'Height': 0.019912945106625557,
                               'Left': 0.20923098921775818,
                               'Top': 0.20798759162425995,
                               'Width': 0.188892662525177},
               'Polygon': [{'X': 0.20923098921775818, 'Y': 0.20798759162425995},
                           {'X': 0.3981236517429352, 'Y': 0.20798759162425995},
                           {'X': 0.3981236517429352, 'Y': 0.22790053486824036},
                           {'X': 0.20

  'RowIndex': 12,
  'RowSpan': 1},
 {'BlockType': 'CELL',
  'ColumnIndex': 3,
  'ColumnSpan': 1,
  'Confidence': 99.99972534179688,
  'Geometry': {'BoundingBox': {'Height': 0.0505482442677021,
                               'Left': 0.3981236517429352,
                               'Top': 0.5250629186630249,
                               'Width': 0.2543957531452179},
               'Polygon': [{'X': 0.3981236517429352, 'Y': 0.5250629186630249},
                           {'X': 0.6525194048881531, 'Y': 0.5250629186630249},
                           {'X': 0.6525194048881531, 'Y': 0.5756111741065979},
                           {'X': 0.3981236517429352, 'Y': 0.5756111741065979}]},
  'Id': '730c2551-0387-41d2-b823-8e3b7f07b825',
  'Relationships': [{'Ids': ['db1486df-86d4-416c-89dd-5b66f2c90bdf',
                             'a270e4d9-d4b5-4c1e-991e-70fdbdeca7f8',
                             '1197576d-9789-4357-be87-588b4206ca6c',
                             '7cc18232-bb8f-4bb6-815a-dd

# EXTRACTING KEY VALUE PAIRS/JSON FILE OUT OF CSV FILE